Persistent Cell Demo
=====================

This notebook demonstrates how to specify, execute and visualize a CC3D model of persistent cell migration. 

Basic Setup
------------------

An interactive CC3D simulation can be initialized from a list of core specs. 
Start a list of core specs that define the simulation by specifying a 
two-dimensional simulation with a 100x100 lattice and second-order Potts neighborhood.

In [ ]:
from math import sqrt
from cc3d.core.PyCoreSpecs import Metadata, PottsCore, CellTypePlugin, VolumePlugin, VolumeEnergyParameter, SurfacePlugin, SurfaceEnergyParameter, CenterOfMassPlugin

specs = [
    Metadata(), 
    PottsCore(dim_x=100, dim_y=100, neighbor_order=2, boundary_x='Periodic', boundary_y='Periodic'),
    CellTypePlugin("Cell"),
    VolumePlugin(VolumeEnergyParameter("Cell", 36, 2)),
    SurfacePlugin(2, None, SurfaceEnergyParameter("Cell", 60, 2)),
    CenterOfMassPlugin()
]

Cell Distribution Initialization
---------------------------------

Initialize one cell. 

In [ ]:
from cc3d.core.PySteppables import SteppableBasePy

class InitializerSteppable(SteppableBasePy):

    def __init__(self, *args, **kwargs):
        
        super().__init__(*args, **kwargs)
        
        self.cell = None
    
    def start(self):
        
        cell_area = self.specs.volume['Cell'].target_volume
        cell_hwidth = int(sqrt(cell_area))
        
        self.cell = self.new_cell(self.cell_type.Cell)
        self.cell_field[self.dim.x // 2 - cell_hwidth:self.dim.x // 2 + cell_hwidth,
                        self.dim.y // 2 - cell_hwidth:self.dim.y // 2 + cell_hwidth, 
                        0] = self.cell
        
        pvec = self.persistence_plugin.getModel(self.cell).persistenceVector()
        print(0, [pvec.x, pvec.y, pvec.z])

    def step(self, mcs):
        pvec = self.persistence_plugin.getModel(self.cell).persistenceVector()
        print(mcs, [pvec.x, pvec.y, pvec.z])

steppable = InitializerSteppable()

# Persistence Cell Motion

Add static persistent motion along the X-domain.

In [ ]:
from cc3d.core.PyCoreSpecs import PersistencePlugin, PersistencePluginInitTransformRotate

persistence = PersistencePlugin()
model = persistence.new_model_an(cell_type="Cell", 
                                 magnitude=10.0,
                                 force_mode="Extension",
                                 work_term="Regular",
                                 vector_init=[PersistencePluginInitTransformRotate(180.0, axis='Z')]
                                 )

# model = persistence.new_model_sr(cell_type="Cell",
#                                  magnitude=20.0,
#                                  force_mode="Extension",
#                                  work_term="Regular",
#                                  vector_init=[PersistencePluginInitTransformRotate(180.0, axis='Z')],
#                                  period=50
#                                  )

# model = persistence.new_model_an(cell_type="Cell", 
#                                  magnitude=5.0,
#                                  force_mode="Reciprocal",
#                                  work_term="Mass",
#                                  vector_init=[PersistencePluginInitTransformRotate(180.0, axis='Z')],
#                                  stdev3=0.09
#                                  )

specs.append(persistence)

Simulation Launch
------------------

Initialize a CC3D simulation service instance and register all simulation specification. 

In [ ]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService

cc3d_sim = CC3DSimService()
cc3d_sim.register_specs(specs)
cc3d_sim.register_steppable(steppable)
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

Visualization
--------------

Show a single frame to visualize simulation data as it is generated. 

In [ ]:
from IPython.display import display

win = cc3d_sim.visualize(plot_freq=10)
win.window_width_percent = 0.25
win.show()
display(cc3d_sim.jupyter_run_button())